Build a regression model.

In [4]:
import numpy as np
import pandas as pd

from sklearn import linear_model, datasets
import statsmodels.api as sm

import sqlite3
from sqlalchemy import create_engine

In [7]:
# Create a connection to the SQLite database
engine = create_engine('sqlite:///../data/bike_station_pois.db')

# Define the SQL query to extract relevant fields and calculate PF
SQL = """
SELECT 
    *
FROM joined_pois
"""

# Read the query result into a Pandas DataFrame
pois = pd.read_sql_query(SQL, engine)

# Display the resulting DataFrame
print(pois)


      query_latitude  query_longitude  num_categories  \
0          43.685924       -79.376304               1   
1          43.685924       -79.376304               3   
2          43.685924       -79.376304               1   
3          43.685924       -79.376304               1   
4          43.685924       -79.376304               3   
...              ...              ...             ...   
4647       43.649192       -79.383632               1   
4648       43.649192       -79.383632               2   
4649       43.649192       -79.383632               3   
4650       43.649192       -79.383632               1   
4651       43.649192       -79.383632               2   

                                  name     distance  num_attributes  \
0                    Rosedale's Finest    27.000000               0   
1                         Dolce Bakery    38.000000               0   
2                 Kirkwood's Rib Joint   876.000000               0   
3              Picnic Cafe & Wi

In [6]:
# Define the SQL query to extract relevant fields and calculate PF
SQL = """
SELECT 
    *
FROM bike_stations
"""

# Read the query result into a Pandas DataFrame
bike_stations = pd.read_sql_query(SQL, engine)

bike_stations = bike_stations.drop(columns = 'Unnamed: 0')

# Display the resulting DataFrame
print(bike_stations)


     Unnamed: 0   latitude  longitude  free_bikes
0             0  43.685924 -79.376304           0
1             1  43.667763 -79.308117          12
2             2  43.691468 -79.288619           2
3             3  43.640722 -79.391051           2
4             4  43.643834 -79.396649          13
..          ...        ...        ...         ...
872         872  43.645209 -79.396074          18
873         873  43.683378 -79.322961           4
874         874  43.658988 -79.438715           1
875         875  43.683351 -79.415620           0
876         876  43.655351 -79.383460          33

[877 rows x 4 columns]


In [16]:
alldata = pd.merge(
    left = pois, 
    right = bike_stations,
    how = 'left',
    left_on = ['query_latitude', 'query_longitude'],
    right_on = ['latitude', 'longitude']
)

alldata = alldata.drop(columns = ['longitude', 'latitude'])

alldata.head()

,query_latitude,query_longitude,num_categories,name,distance,num_attributes,source,free_bikes
0,43.685924,-79.376304,1,Rosedale's Finest,27.0,0,foursquare,0
1,43.685924,-79.376304,3,Dolce Bakery,38.0,0,foursquare,0
2,43.685924,-79.376304,1,Kirkwood's Rib Joint,876.0,0,foursquare,0
3,43.685924,-79.376304,1,Picnic Cafe & Wine Shop,897.0,0,foursquare,0
4,43.685924,-79.376304,3,Holly Berry Fair,381.0,0,foursquare,0


In [45]:
grouped_alldata = alldata.groupby(['query_longitude', 'query_latitude', 'source']).agg(
    lat=('query_latitude', 'last'),
    long=('query_longitude', 'last'),
    source=('source', 'last'),
    free_bikes=('free_bikes', 'last'),
    distance=('distance', 'mean'),
    poi_count=('name', 'count'),
    num_attributes=('num_attributes', 'mean'),
    num_categories=('num_categories', 'mean')
)

print(grouped_alldata)


                                                 lat       long      source  \
query_longitude query_latitude source                                         
-79.503016      43.662540      foursquare  43.662540 -79.503016  foursquare   
                               yelp        43.662540 -79.503016        yelp   
-79.500500      43.774100      foursquare  43.774100 -79.500500  foursquare   
                               yelp        43.774100 -79.500500        yelp   
-79.500027      43.760740      foursquare  43.760740 -79.500027  foursquare   
...                                              ...        ...         ...   
-79.229204      43.726902      yelp        43.726902 -79.229204        yelp   
-79.209707      43.745311      foursquare  43.745311 -79.209707  foursquare   
                               yelp        43.745311 -79.209707        yelp   
-79.130120      43.780550      foursquare  43.780550 -79.130120  foursquare   
                               yelp        43.780550

In [47]:
dummies = pd.get_dummies(grouped_alldata['source'], prefix='source', drop_first=True, dtype=int)
print(dummies)

# Concatenate the original DataFrame with the new dummy DataFrame
grouped_alldata = pd.concat([grouped_alldata.drop(columns = 'source'), dummies], axis=1)
print(grouped_alldata)

                                           source_yelp
query_longitude query_latitude source                 
-79.503016      43.662540      foursquare            0
                               yelp                  1
-79.500500      43.774100      foursquare            0
                               yelp                  1
-79.500027      43.760740      foursquare            0
...                                                ...
-79.229204      43.726902      yelp                  1
-79.209707      43.745311      foursquare            0
                               yelp                  1
-79.130120      43.780550      foursquare            0
                               yelp                  1

[100 rows x 1 columns]
                                                 lat       long  free_bikes  \
query_longitude query_latitude source                                         
-79.503016      43.662540      foursquare  43.662540 -79.503016          14   
                        

In [48]:
X = grouped_alldata[['source_yelp', 'distance', 'poi_count', 'num_attributes', 'num_categories']]
y = grouped_alldata[['free_bikes']]

X = sm.add_constant(X) # adding a constant

In [49]:
lin_reg = sm.OLS(y,X)

Provide model output and an interpretation of the results. 

In [50]:
model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     2.358
Date:                Sat, 26 Apr 2025   Prob (F-statistic):             0.0461
Time:                        16:29:51   Log-Likelihood:                -341.68
No. Observations:                 100   AIC:                             695.4
Df Residuals:                      94   BIC:                             711.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              2.0399      8.904      0.

Based on the adjusted R2 of this model (0.064), it is not a good model to predict the number of free bikes based on the presence and quality of information on restaurants, bars, or cafes nearby. The only signficant variable in the model is the average distance between pois and the bike station, and the coefficient suggests that as average distance increases the number of free bikes decreases. However, if you refer to my discussion in the joining_data file, I mention that distance may represent different things in the Yelp or Foursquare data (straight-line verus path-analysis), which may impact findings. I re-ran the model with only distance (and I kept the source variable as I felt excluding it could also bias the results as you would have two data points per query long/lat) and the adjusted R2 did not improve much.

Of note, I used aggregate calculations per query as I felt that it better reflected the quality of data per query around each bike station and wanted to avoid potential correlation where each bike station had multiple data points that may not have been independent. Also I did not remove duplicates between queries (e.g. if station A & B had the same restaurant in their 1000 m radius, which could have been identified with the poi ID) as I felt it would take away from the information I could gather of the quality of data around each query and how that same poi may contribute to the number of free bikes at both of the stations. 

In my opinion, these results suggest that there may be more factors contributing to the number of free bikes at each bike station in Toronto than the presence of different restaurants, bars, or cafes in the area. I live in Toronto and from experience can say that certain neighbourhoods are frequented by different types of people who may not enjoy commuting with bikes to go to these kinds of establishments, whereas other neighbourhoods or cities may have different patterns (e.g. in Montreal I know many locals who would enjoy biking home after going to the bar with friends). 

In [62]:
X = grouped_alldata[['source_yelp', 'distance']]
y = grouped_alldata[['free_bikes']]

X = sm.add_constant(X) # adding a constant
lin_reg = sm.OLS(y,X)
model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     3.678
Date:                Sun, 27 Apr 2025   Prob (F-statistic):             0.0580
Time:                        12:10:16   Log-Likelihood:                -345.74
No. Observations:                 100   AIC:                             695.5
Df Residuals:                      98   BIC:                             700.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.2852      2.375      5.172      0.0

# Stretch

How can you turn the regression model into a classification model?

You could turn this regression into a classification model by establishing a binary outcome (e.g. no free bikes v yes free bikes) and rerunning the analysis. It would use the same process of creating a dummy variable as I showed earlier when coding the source variable above. 